In [ ]:
import os
import json
import pandas as pd
from datetime import datetime

# =============================================
# FOLDER FILE JSON
# =============================================
folder_path = "data/"   # ganti sesuai lokasi JSON

# =============================================
# LOAD ALL review*.json
# =============================================
review_files = [f for f in os.listdir(folder_path) if f.startswith("reviews") and f.endswith(".json")]

all_reviews = []

for file_name in review_files:
    with open(os.path.join(folder_path, file_name), "r", encoding="utf-8") as f:
        data = json.load(f)
        reviews_list = data.get("reviews", [])

        for r in reviews_list:
            reviewer = r.get("reviewer", {}).get("displayName", "")
            rating_raw = r.get("starRating", "")

            # Convert rating text → number
            rating_map = {
                "ONE": 1, "TWO": 2, "THREE": 3, "FOUR": 4, "FIVE": 5
            }
            rating = rating_map.get(rating_raw.upper(), "")

            comment = r.get("comment", "")
            create_raw = r.get("createTime", "")

            # Convert ISO date → dd-MMM-yy (3-Jun-25)
            try:
                dt = datetime.fromisoformat(create_raw.replace("Z", ""))
                review_date = dt.strftime("%d-%b-%y")
            except:
                review_date = ""

            # push clean review
            all_reviews.append({
                "Review Date": review_date,
                "User": reviewer,
                "Comment": comment,
                "Rate": rating,
                "Branch": "Canggu",
                "Platform": "Google",
                "Client Name": reviewer,
                "Treatment Date": review_date,
                "Service": "",
            })

df = pd.DataFrame(all_reviews)

# =============================================
# STAFF DICTIONARY
# =============================================
staff_dict = {
    # ===== DOKTER =====
    "dr josie": "dr. Josie",
    "dr. josie": "dr. Josie",
    "josie": "dr. Josie",
    "dr ivan": "dr. Ivan",
    "dr. ivan": "dr. Ivan",
    "ivan": "dr. Ivan",
    "dr yusta": "dr. Yusta",
    "dr. yusta": "dr. Yusta",
    "yusta": "dr. Yusta",
    "dr reva": "dr. Reva",
    "dr. reva": "dr. Reva",
    "reva": "dr. Reva",
    "dr brill": "dr. Brill",
    "dr. brill": "dr. Brill",
    "brill": "dr. Brill",
    "dr theresia": "dr. Theresia",
    "dr. theresia": "dr. Theresia",
    "theresia": "dr. Theresia",
    "dr angel": "dr. Angel",
    "dr. angel": "dr. Angel",
    "dokter angel": "dr. Angel",
    "angel": "dr. Angel",
    "dr eva": "dr. Eva",
    "dr. eva": "dr. Eva",
    "eva": "dr. Eva",
    # ===== STAFF =====
    "prima": "Prima",
    "dita": "Dita",
    "widha": "Widha",
    "rita": "Rita",
    "miranda": "Miranda",
    "krisna": "Krisna",
    "made": "Made",
    "ani": "Ani",
    "rini": "Rini",
    "dinda": "Dinda",
    "fiqih": "Fiqih",
    "tini": "Tini",
    "yumi": "Yumi",
    "sri": "Sri",
    "gauri": "Gauri",
    "kathleen": "Kathleen",
    "ayuk": "Ayuk",
    "arfi": "Arfi",
    "putri": "Putri",
    "bayu": "Bayu",
    "winda": "Winda",
    "julie": "Julie",
    "tiwi": "Tiwi",
    "gede": "Gede",
    "ria": "Ria",
    # variasi weby
    "weby": "Weby",
    "webi": "Weby",
    "webby": "Weby",
    "santi": "Santi",
    # Driver
    "driver made": "Driver Made",
    "made driver": "Driver Made",
    "driver": "Driver Made",
    "aura": "Aura",
    "ima": "Ima",
    "surya": "Surya",
    "erdin": "Erdin",
    "amel": "Amel",
    "novi": "Novi"
}


def extract_staff(text):
    if not isinstance(text, str):
        return ["", "", ""]
    t = text.lower()
    found = []
    for key, name in staff_dict.items():
        if key in t:
            found.append(name)
    found = list(dict.fromkeys(found))[:3]
    found += [""] * (3 - len(found))
    return found

df["staff_1"], df["staff_2"], df["staff_3"] = zip(*df["Comment"].apply(extract_staff))

# service_1 mengikuti kolom Service
df["service_1"] = df["Service"]

# =============================================
# SIMPAN CSV OUTPUT FINAL
# =============================================
output_path = "FINAL_google_reviews_clean.csv"
df.to_csv(output_path, index=False, encoding="utf-8-sig")

output_path


'FINAL_google_reviews_clean.csv'

In [ ]:
service_dict = {
    "hydrafacial": "Hydrafacial",
    "360 hydrafacial": "Cocoon Signature 360 Hydrafacial",
    "laser hair removal": "Laser Hair Removal",
    "microdermabrasion": "Diamond Microdermabrasion",
    "diamond microdermabrasion": "Diamond Microdermabrasion",
    "botox": "Botox",
    "filler": "Filler",
    "facial": "Facial",
    "colon hydrotherapy": "Colon Hydrotherapy",
    "prp": "PRP Treatment"
}
def extract_service(text):
    if not isinstance(text, str):
        return ""
    t = text.lower()
    for key, service in service_dict.items():
        if key in t:
            return service
    return ""

df = pd.read_csv("FINAL_google_reviews_clean.csv")

df["service_1"] = df["Comment"].apply(extract_service)

df.to_csv("FINAL_google_reviews_with_service.csv", index=False, encoding="utf-8-sig")
service_count = (
    df[df["service_1"] != ""]
    .groupby("service_1")
    .size()
    .reset_index(name="total_mentions")
    .sort_values("total_mentions", ascending=False)
)

service_count.to_excel("service_count_summary.xlsx", index=False)

